In [49]:
!pip install selenium

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm import tqdm
import os
from selenium import webdriver  

### Выгружаем дату и ссылку

In [13]:
url_ = 'https://www.rbc.ru/newspaper/'
res = pd.DataFrame()
t0 = time.time()

    
for y in range(2014,2021):
    
        for m in range(1,13):
            
            for d in range(1,32):

                    st_m = str(m)
                    if len(st_m) == 1: st_m = '0'+st_m

                    st_d = str(d)
                    if len(st_d) == 1: st_d = '0'+st_d

                    st = str(y) + '/' + st_m + '/' + st_d

                    url = url_ + st

                    try: 
                        page = requests.get(url)
                    except:
                        r = {'step':st, 'error':'downloading error', 'time':(time.time()- t0)}
                        res = res.append(pd.DataFrame(r, index = [len(res)]))
                        continue
                    
                    

                    try:
                        p = BeautifulSoup(page.content, "html.parser")
                        l_1 = p.find_all('a', {'class' : 'newspaper-page__news'})
                        l_2 = p.find_all('a', {'class' : 'newspaper-section__group__news__link'})
                        l_3 = p.find_all('a', {'class' : 'newspaper-section__opinion__author-block'})
                        
                    except:
                        r = {'step':st, 'error':'parsing error', 'time':(time.time()- t0)}
                        res = res.append(pd.DataFrame(r, index = [len(res)]))
                    


                    for x in l_1 + l_2 + l_3:
                        r = {'step':st, 'url':x.get('href'), 'time':(time.time()- t0)}
                        res = res.append(pd.DataFrame(r, index = [len(res)]))

                    print(st, end = '\r')
        res.to_csv('rbc_url_0.csv', sep='|')


res.to_csv('rbc_url_0.csv', sep='|')
res.to_excel('rbc_url_0.xlsx')
res.head()

,step,url,time
0,2014/01/13,https://www.rbc.ru/newspaper/2014/01/13/56bfb8...,2.637483
1,2014/01/13,https://www.rbc.ru/newspaper/2014/01/13/56bfb8...,2.638479
2,2014/01/13,https://www.rbc.ru/newspaper/2014/01/13/56bfb6...,2.640478
3,2014/01/13,https://www.rbc.ru/newspaper/2014/01/13/56bfb6...,2.642470
4,2014/01/13,https://www.rbc.ru/newspaper/2014/01/13/56bfb8...,2.643500


In [15]:
res = res.drop_duplicates(['url'])
res.to_csv('rbc_url_0.csv', index = False, sep='|')
res.to_excel('rbc_url_0.xlsx')
url_list = res.copy()
f = url_list['url'].tolist()
res.head()

,step,url,time
0,2014/01/13,https://www.rbc.ru/newspaper/2014/01/13/56bfb8...,2.637483
1,2014/01/13,https://www.rbc.ru/newspaper/2014/01/13/56bfb8...,2.638479
2,2014/01/13,https://www.rbc.ru/newspaper/2014/01/13/56bfb6...,2.640478
3,2014/01/13,https://www.rbc.ru/newspaper/2014/01/13/56bfb6...,2.642470
4,2014/01/13,https://www.rbc.ru/newspaper/2014/01/13/56bfb8...,2.643500


In [21]:
res = pd.DataFrame()
options = webdriver.ChromeOptions()
options.add_argument("user-data-dir=C:\\Users\\ВТБ\\AppData\\Local\\Google\\Chrome\\User Data\\Default")
#br = webdriver.Chrome(os.getcwd() + '\\chromedriver.exe', chrome_options=options) 
#C:/Users/ВТБ/Desktop/hse_news_project-master
br = webdriver.Chrome('C:/Users/ВТБ/Desktop/hse_news_project-master/chromedriver.exe', chrome_options=options) 

for x_ in range(len(f)):
    url = f[x_]
    
    try:
        #page = urllib2.urlopen(url)
        br.get(url)
        page = br.page_source
    except:
        r = {'url':x, "error":"downloading error"}
        res = res.append(pd.DataFrame(r, index = [len(res)]))
        continue
        
    #page = br.page_source
    p = BeautifulSoup(page)
    
    # текст статьи
    try:
        p_ = p.find('div', {'class':'article__text'})
        st = ''
        for x in p_.find_all('p'): st = st + x.text + ' '
    except:
        st = None
    
    try:
        p_ = p.find_all('div', {'class':"article__header__info-block"})[0]
        category = p_.find('a', {'class':"article__header__category"}).text
    except:
        category = None
        
    try:    
        date = p_.find('span', {'class':"article__header__date"}).text.replace('\n', '')
    except:
        date = None
    
    try: 
        article_header = p.find('div', {'class':'article__header__title'}).text.replace('\n', '')
    except:
        article_header = None
    
    try:
        article_subheader = p.find('div', {'class':'article__text__overview'}).text.replace('\n', '')
    except:
        article_subheader = None


    r = {'url': url, "text":st, "article_header":article_header, "artical_subheader":article_subheader,
         "category":category, "date":date}
    res = res.append(pd.DataFrame(r, index = [len(res)]))
    
    if x_ % 100 == 0 or x_ == (len(f) - 1): 
        res.to_csv('rbc_texts_1.csv', sep='|')
        print(x_, end = '\r')
        
res.to_csv('rbc_texts_1.csv', sep='|')
res.to_excel('rbc_texts_1.xlsx')
res.head()


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  


,url,text,article_header,artical_subheader,category,date
0,https://www.rbc.ru/newspaper/2014/01/13/56bfb8...,\n\tВ ходе весенней сессии Госдума рассмотрит ...,Дорога не повод для дискуссий,None,"Общество,","13 янв 2014, 01:05"
1,https://www.rbc.ru/newspaper/2014/01/13/56bfb8...,"\n\tКак стало известно РБК daily, «дочка» Сбер...",«Сетелем» подкатит к дилерам,None,"Общество,","13 янв 2014, 01:05"
2,https://www.rbc.ru/newspaper/2014/01/13/56bfb6...,"\n\t«Роснефть», которая в прошлом году приобре...","1,5 миллиарда за имя",None,"Общество,","13 янв 2014, 01:05"
3,https://www.rbc.ru/newspaper/2014/01/13/56bfb6...,"\n\tКомпании, ведущие бизнес по офшорным схема...",Отцовская забота,None,"Общество,","13 янв 2014, 01:05"
4,https://www.rbc.ru/newspaper/2014/01/13/56bfb8...,\n\tСенаторы подготовили и готовятся внести в ...,Эксперты не при делах,None,"Общество,","13 янв 2014, 01:05"
